In [136]:
import itertools
import nltk
import operator
import nltk
nltk.download('wordnet')
from math import log
from matplotlib import pyplot as plt
from string import punctuation
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.probability import FreqDist
from collections import Counter
from nltk.corpus import stopwords


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tifat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [137]:
# Functions to normalize the texts
def remove_stopwords(words):
    stop_words = set(stopwords.words('english'))
    filtered_words = [w for w in words if not w in stop_words]
    return filtered_words

def stemming(words):
    return [SnowballStemmer("english").stem(word) 
            for word in words]

def remove_punctuation(text_file):
    words = nltk.word_tokenize(text_file.lower())
    words = [word for word in words if word.isalpha()]
    
    return words

def freq_dist(words):
    freq_dist = nltk.FreqDist(words)
    return freq_dist


# Function to classify the labels (senses) of the disambiguated words and their related words distributions      
def get_sentence_and_label(fname):
    tags = []
    desc = []
    with open(fname) as f:
        while True:
            tags.append(f.readline())
            desc.append(f.readline())
            spc = f.readline()
            if not spc:break #EOF
    tags = [x.strip().lower().rpartition('%')[2] for x in tags]
    desc = [x.strip().lower().rpartition('%')[2] for x in desc]
    word_list = []
    
    for i in range(0,2):
        words = remove_punctuation(desc[i])
#         print(words)
        words = remove_stopwords(words)
#         print(words)
        words = stemming(words)
        words = set(words)
        word_list.append(words)
#         print(type(words))
#         print(words)
#         freq_dist[i] = freq_dist(words)
    print('List of senses and their related words for ' + str(fname) + ' (After Normalization):')
    print('\n')
    print(tags)
    print(word_list)
    return tags, word_list



In [138]:
# Function returns the true classification tags and the sentences list for a word
def get_test_sentence_and_label(fname):
    tags = []
    desc = []
    with open(fname) as f:
        while True:
            tags.append(f.readline())
            desc.append(f.readline())
            spc = f.readline()
            if not spc:break #EOF
    tags = [x.strip().lower().rpartition('%')[2] for x in tags]
    desc = [x.strip().lower().rpartition('%')[2] for x in desc]
    word_list = []
    
    for i in range(len(desc)):
        words = remove_punctuation(desc[i])
#         print(words)
        words = remove_stopwords(words)
#         print(words)
        words = stemming(words)
        words = set(words)
        word_list.append(words)
#         print(type(words))
#         print(words)
#         freq_dist[i] = freq_dist(words)
    
#     print(word_list)
    return tags, word_list

In [139]:
# Reading the files
bass_def = "wsd_data/bass.definition"
crane_def = "wsd_data/crane.definition"
motion_def = "wsd_data/motion.definition"
palm_def = "wsd_data/palm.definition"
plant_def = "wsd_data/plant.definition"
tank_def = "wsd_data/tank.definition"

bass_test = "wsd_data/bass.test"
crane_test = "wsd_data/crane.test"
motion_test = "wsd_data/motion.test"
palm_test = "wsd_data/palm.test"
plant_test = "wsd_data/plant.test"
tank_test = "wsd_data/tank.test"

In [140]:
# Function calling to classify the lables (senses) for a words
[bass_tag, bass_desc] = get_sentence_and_label(bass_def)
[crane_tag, crane_desc] = get_sentence_and_label(crane_def)
[motion_tag, motion_desc] = get_sentence_and_label(motion_def)
[palm_tag, palm_desc] = get_sentence_and_label(palm_def)
[plant_tag, plant_desc] = get_sentence_and_label(plant_def)
[tank_tag, tank_desc] = get_sentence_and_label(tank_def)

# Function calling to get the true classification and sentences for each words
[bass_test_tag, bass_ctx] = get_test_sentence_and_label(bass_test)
[crane_test_tag, crane_ctx] = get_test_sentence_and_label(crane_test)
[motion_test_tag, motion_ctx] = get_test_sentence_and_label(motion_test)
[palm_test_tag, palm_ctx] = get_test_sentence_and_label(palm_test)
[plant_test_tag, plant_ctx] = get_test_sentence_and_label(plant_test)
[tank_test_tag, tank_ctx] = get_test_sentence_and_label(tank_test)



List of senses and their related words for wsd_data/bass.definition (After Normalization):


['music', 'fish']
[{'sever', 'guitar', 'refer', 'composit', 'one', 'describ', 'instrument', 'vocal', 'music', 'sing', 'part', 'rang', 'low', 'bass', 'voic', 'harmoni', 'also', 'lowest', 'frequenc', 'may', 'type', 'classic', 'male', 'tone'}, {'speci', 'fish', 'belong', 'middl', 'order', 'perch', 'bar', 'freshwat', 'share', 'larg', 'mean', 'english', 'bass', 'word', 'come', 'mani', 'name', 'term', 'differ', 'encompass', 'marin', 'perciform'}]
List of senses and their related words for wsd_data/crane.definition (After Normalization):


['machine', 'bird']
[{'place', 'rope', 'thing', 'crane', 'sheav', 'lift', 'horizont', 'general', 'heavi', 'equip', 'materi', 'wire', 'use', 'lower', 'transport', 'main', 'type', 'chain', 'machin', 'move', 'hoist'}, {'speci', 'live', 'genera', 'heron', 'pull', 'antarctica', 'unlik', 'larg', 'except', 'bird', 'south', 'fifteen', 'america', 'fli', 'group', 'back', 'gru

In [141]:
# Function for lesk prediction score
def lesk_pred(set_x, set_y):
#     set_x = set(x)
#     set_y = set(y)
    intersect = set_x.intersection(set_y)
    score = (2 * len(intersect)) / (len(set_x) + len(set_y))
    return score

In [142]:
# Function to caculate jaccard simillarity
def jaccard_sim(set_x, set_y):
    intersect = set_x.intersection(set_y)
    union = set_x.union(set_y)
    
    score = (2 * len(intersect)) / len(union)
    return score

In [143]:
# Function to compute the accuracy
def compute_accuracy(true_list, predict_list):
    correct = 0
    for i in range(len(true_list)):
        if true_list[i] == predict_list[i]:
            correct += 1
    
    accuracy = correct / len(true_list)
    return accuracy
    
    

In [133]:
# lex_predict_list = []
# jacc_predict_list = []
# most_freq_predict_list = []
# for i in range(len(bass_test_tag)):
#     # For Lesk Algorithm 
#     class_1 = lesk_pred(bass_ctx[i], bass_desc[0])
#     class_2 = lesk_pred(bass_ctx[i], bass_desc[1])
    
#     if class_1 >= class_2:
#         lex_predict_list.append(bass_tag[0])
#     else:
#         lex_predict_list.append(bass_tag[1])
    
#     class_1 = jaccard_sim(bass_ctx[i], bass_desc[0])
#     class_2 = jaccard_sim(bass_ctx[i], bass_desc[1])
    
#     if class_1 >= class_2:
#         jacc_predict_list.append(bass_tag[0])
#     else:
#         jacc_predict_list.append(bass_tag[1])
    
#     most_freq_predict_list.append(bass_tag[0])
        
# print(type(lex_predict_list))
# print('Accuracy using Lesk Algorithm: ', compute_accuracy(bass_test_tag, lex_predict_list))
# print('Accuracy using Jaccard Algorithm: ', compute_accuracy(bass_test_tag, jacc_predict_list))
# print('Accuracy by assigning most Frequent: ', compute_accuracy(bass_test_tag, most_freq_predict_list))

In [144]:
# Function to calculate the accuracy for each of the algorithm for a disambigious word
def calculate_accuracy(word, class_tag, class_desc, test_tag, test_desc):
    lex_predict_list = []
    jacc_predict_list = []
    most_freq_predict_list = []
    for i in range(len(test_tag)):
        # For Lesk Algorithm 
        class_1 = lesk_pred(test_desc[i], class_desc[0])
        class_2 = lesk_pred(test_desc[i], class_desc[1])

        if class_1 >= class_2:
            lex_predict_list.append(class_tag[0])
        else:
            lex_predict_list.append(class_tag[1])

        class_1 = jaccard_sim(test_desc[i], class_desc[0])
        class_2 = jaccard_sim(test_desc[i], class_desc[1])

        if class_1 >= class_2:
            jacc_predict_list.append(class_tag[0])
        else:
            jacc_predict_list.append(class_tag[1])

        most_freq_predict_list.append(class_tag[0])

    print('For the word,', word, ':')
    print('Accuracy using Lesk Algorithm: ', compute_accuracy(test_tag, lex_predict_list))
    print('Accuracy using Jaccard Algorithm: ', compute_accuracy(test_tag, jacc_predict_list))
    print('Accuracy by assigning most Frequent: ', compute_accuracy(test_tag, most_freq_predict_list))
    print('\n')

In [146]:
calculate_accuracy('Bass', bass_tag, bass_desc, bass_test_tag, bass_ctx)
calculate_accuracy('Crane', crane_tag, crane_desc, crane_test_tag, crane_ctx)
calculate_accuracy('Motion', motion_tag, motion_desc, motion_test_tag, motion_ctx)
calculate_accuracy('Palm', palm_tag, palm_desc, palm_test_tag, palm_ctx)
calculate_accuracy('Plant', plant_tag, plant_desc, plant_test_tag, plant_ctx)
calculate_accuracy('Tank', tank_tag, tank_desc, tank_test_tag, tank_ctx)


For the word, Bass :
Accuracy using Lesk Algorithm:  0.5833333333333334
Accuracy using Jaccard Algorithm:  0.5833333333333334
Accuracy by assigning most Frequent:  0.8981481481481481


For the word, Crane :
Accuracy using Lesk Algorithm:  0.8315789473684211
Accuracy using Jaccard Algorithm:  0.8315789473684211
Accuracy by assigning most Frequent:  0.7578947368421053


For the word, Motion :
Accuracy using Lesk Algorithm:  0.6683168316831684
Accuracy using Jaccard Algorithm:  0.6683168316831684
Accuracy by assigning most Frequent:  0.7029702970297029


For the word, Palm :
Accuracy using Lesk Algorithm:  0.7920792079207921
Accuracy using Jaccard Algorithm:  0.7920792079207921
Accuracy by assigning most Frequent:  0.7079207920792079


For the word, Plant :
Accuracy using Lesk Algorithm:  0.5925925925925926
Accuracy using Jaccard Algorithm:  0.5925925925925926
Accuracy by assigning most Frequent:  0.5396825396825397


For the word, Tank :
Accuracy using Lesk Algorithm:  0.7376237623762376